In [3]:
%load_ext autoreload
%autoreload 2

from NewsContent import *
from UserContent import *
from preprocessing import *
from PEGenerator import *
import PEGenerator
from models import *
from utils import *
from Encoders import *

import os
import numpy as np
import json
import random

In [4]:
data_root_path = "../data/Challenge/"
embedding_path = None
KG_root_path = None
popularity_path = "../data/Challenge/popularity"
config = {'title_length':30,
              'body_length':100,
              'max_clicked_news':50,
              'npratio':1,
              'news_encoder_name':"CNN",
              'user_encoder_name':"Att",
             'attrs':['title','vert','entity'],
             'word_filter':0,
             'data_root_path':data_root_path,
             'embedding_path':embedding_path,
             'KG_root_path':KG_root_path,
            'popularity_path':popularity_path,
             'max_entity_num':5}
model_config = {
        'news_encoder':1,
        'popularity_user_modeling':True,
        'rel':True,
        'ctr':True,
        'content':True,
        'rece_emb':True,
        'activity':True

    }

In [5]:
News = NewsContent(config)
TrainUsers = UserContent(News.news_index,config,'train.tsv',2)
ValidUsers = UserContent(News.news_index,config,'val.tsv',1)
TestUsers = UserContent(News.news_index,config,'test.tsv',2)

19779
2472
2473


In [6]:
train_sess,train_buckets, train_user_id, train_label = get_train_input(TrainUsers.session,News.news_index,config)
test_impressions, test_userids = get_test_input(TestUsers.session,News.news_index)
val_impressions, val_userids = get_test_input(ValidUsers.session,News.news_index)

19779
19916
2473
2472


In [16]:
print(train_user_id)

[    0     1     2 ... 19776 19777 19778]


In [7]:
title_word_embedding_matrix, have_word = load_matrix(embedding_path,News.word_dict)

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [21]:
# train_generator = TrainGenerator(News,TrainUsers,train_sess,train_user_id,train_buckets,train_label,32)
# test_user_generator = UserGenerator(News,TestUsers,32)
# val_user_generator = UserGenerator(News,ValidUsers,32)
# news_generator = NewsGenerator(News,32)


print(sum(sum(News.news_stat_click)))

24888.0


In [ ]:
class EbnerdDataSet(Dataset):
    def __init__(self, News, User, session, user_id, buckets, labels, batch_size):
        self.News = News
        self.User = User
        self.session = session
        self.user_id = user_id
        self.buckets = buckets
        self.labels = labels
        self.batch_size = batch_size
        self.length = len(self.session)
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        candidates = self.session[idx]
        candidates_ctr = News.load_ctr(candidates)
        candidates_rece_emb_index = ...
        user_activity_input = ...
        clicked_input = TrainUsers.click[self.user_id[idx]]
        clicked_ctr = NewsContent.load_ctr(self.clicked_input)
        labels = self.labels[idx]

        input = [candidates,candidates_ctr,candidates_rece_emb_index,user_activity_input,clicked_input,clicked_ctr]
        
        # batch_user_id = self.user_id[idx]
        # batch_buckets = self.buckets[idx]
        # batch_label = self.labels[idx]
        # batch_user = self.User.get_user(batch_user_id)
        # batch_news = self.News.get_news(batch_buckets)
        
        return input, labels



In [10]:
from torch.optim import Adam

# model,user_encoder,news_encoder,bias_news_encoder,bias_content_scorer,scaler,time_embedding_layer,activity_gater = \
# create_pe_model(News, config, model_config, title_word_embedding_matrix, entity_embedding_matrix=None)

train_loader = DataLoader(TrainUsers, batch_size=32, shuffle=True)



for x, y in train_loader:
    print(x)
    print(y)
    break

TypeError: object of type 'UserContent' has no len()

In [1]:


# Step 2: Create your Adam optimizer
optimizer = Adam(model.parameters(), lr=0.0001)

loss_fn = nn.CrossEntropyLoss()

# Step 3: Iterate over the data for the number of epochs
for epoch in range(1):

    # Step 4: Iterate over each batch of data and compute the scores using the forward pass of the network
    model.train()
    for x, y in train_loader:
        optimizer.zero_grad()
        out = model(x)
    
        # Step 5: Compute the lambda gradient values for the pairwise loss (spedup) with the compute_lambda_i method on the scores and the output labels
        loss = loss_fn(out, y)

        # Step 6: Bacward from the scores with the use of the lambda gradient values
        if loss is not None:
            torch.autograd.backward(out, loss)
            
            # Step 7: Update the weights using the optimizer
            optimizer.step()

    # Step 8: At the end of the epoch, evaluate the model on the data using the evaluate_model function (both train and val)
    model.eval()
    train_metrics = evaluate_model(data, model.forward, 'train')
    val_metrics = evaluate_model(data, model.forward, 'validation')
    
        
    print(f"Epoch {epoch+1}/{params.epochs} - Train Metrics: {train_metrics}, Val Metrics: {val_metrics}")

    # Step 9: Append the metrics to train_metrics_epoch and val_metrics_epoch
    train_metrics_epoch.append(train_metrics)
    val_metrics_epoch.append(val_metrics)
    
return {
    "metrics_train": train_metrics_epoch,
    "metrics_val": val_metrics_epoch
}

ModuleNotFoundError: No module named 'Adam'

In [ ]:
# model.compile(loss=['categorical_crossentropy'],
#                   optimizer=Adam(lr=0.0001), 
#                   metrics=['acc'])

In [ ]:
for i in range(10):

    model,user_encoder,news_encoder,bias_news_encoder,bias_content_scorer,scaler,time_embedding_layer,activity_gater = create_pe_model(config,model_config,News,title_word_embedding_matrix,News.entity_embedding)
    model.fit_generator(train_generator,epochs=2)
    news_scoring = news_encoder.predict_generator(news_generator,verbose=True)
    user_scoring = user_encoder.predict_generator(test_user_generator,verbose=True)
    val_user_scoring = user_encoder.predict_generator(val_user_generator,verbose=True)


    news_bias_vecs = bias_news_encoder.predict_generator(news_generator,verbose=True)

    if model_config['content'] and not model_config['rece_emb']:
        bias_candidate_score = bias_content_scorer.predict(news_bias_vecs,batch_size=32,verbose=True)
        bias_candidate_score = bias_candidate_score[:,0]
    else:
        bias_candidate_score = 0

    ctr_weight = scaler.get_weights()[0][0,0]
    time_embedding_matrix = time_embedding_layer.get_weights()[0]
    
    predicted_activity_gates = activity_gater.predict(user_scoring,verbose=True)
    predicted_activity_gates = predicted_activity_gates[:,0]
    
    val_predicted_activity_gates = activity_gater.predict(val_user_scoring,verbose=True)
    val_predicted_activity_gates = val_predicted_activity_gates[:,0]
    
    rankings = news_ranking(model_config,ctr_weight,predicted_activity_gates,user_scoring,news_scoring,
                                bias_candidate_score,news_bias_vecs,time_embedding_matrix,bias_content_scorer,
                                News,test_impressions)
    
    val_rankings = news_ranking(model_config,ctr_weight,val_predicted_activity_gates,val_user_scoring,news_scoring,
                               bias_candidate_score,news_bias_vecs,time_embedding_matrix,bias_content_scorer,
                               News,val_impressions)
    
    performance = evaluate_performance(rankings,test_impressions)
    val_performance = evaluate_performance(val_rankings,val_impressions)

    cold = []
    for TOP_COLD_NUM in [0,1,3,5,]:
        g = evaluate_cold_users(rankings,test_impressions,TestUsers.click,TOP_COLD_NUM)
        cold.append(g)
    diversity = []
    for TOP_DIVERSITY_NUM in range(1,11):
        div_top = evaluate_diversity_topic_all(TOP_DIVERSITY_NUM,rankings,test_impressions,News,TestUsers)
        div_ilxd = evaluate_density_ILxD(TOP_DIVERSITY_NUM,rankings,test_impressions,news_scoring)
        diversity.append([div_top,div_ilxd])
